In [67]:
import argparse
import os
import yaml



In [73]:
stream = open("test.yaml", 'r')
dictionary = yaml.load(stream)
for key, value in dictionary.items():
    print (key, value)
    old, old_aa, new_aa, resid = value

107 [50, 'ASP', 'ALA', '39']
108 [51, 'ALA', 'ASP', '39']
109 [56, 'LYS', 'ALA', '27']
110 [57, 'ALA', 'LYS', '27']
111 [58, 'ARG', 'ALA', '59']
112 [59, 'ALA', 'ARG', '59']
113 [60, 'ARG', 'GLN', '83']
114 [61, 'GLN', 'ARG', '83']
115 [62, 'ARG', 'ALA', '87']
116 [63, 'ALA', 'ARG', '87']


/home/zhangi/miniconda3/envs/perses-sims-oct2020/lib/python3.7/site-packages/ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


In [74]:
# Define mutation parameters
old = 50
new = 107
old_aa = "ASP"
new_aa = "ALA"
resid = "39"
template_file = "/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/18_bnbs_rest/run_rest2_template.sh"
out_dir = "/data/chodera/zhangi/perses_benchmark/neq/12/"

# Define job parameters
wall_time = 12
memory = 5
script_version = 4

In [43]:
# # Read filename
# parser = argparse.ArgumentParser(description='edit tleap file')
# parser.add_argument('out_name', type=str, help='name of out file to check')
# parser.add_argument('in_name', type=str, help='name of in file to edit')
# args = parser.parse_args()



In [75]:
# Edit template bash file
with open(template_file, "r") as f:
    lines_out = f.readlines()

lines_new = []
for line in lines_out:
    if "#BSUB -W" in line:
        line = line[:9] + str(wall_time) + line[11:]
    elif "#BSUB -o" in line:
        line = line[:12] + str(new) + ".%I.out\n"
    elif "#BSUB -eo" in line:
        line = line[:13] + str(new) + ".%I.stderr\n"
    elif "#BSUB -n" in line:
        line = line[:26] + str(memory) + line[27:] 
    elif "#BSUB -J" in line:
        line = line[:13] + str(new) + '[1-4]"\n'
    elif "old=" in line:
        line = f"old={old}\n"
    elif "new=" in line:
        line = f"new={new}\n"
    elif "old_aa=" in line:
        line = f"old_aa={old_aa}\n"
    elif "new_aa=" in line:
        line = f"new_aa={new_aa}\n"
    elif "resid=" in line:
        line = f"resid={resid}\n"
    elif "python generate_rest2_cache" in line:
        line = line[:51] + str(script_version) + line[52:]
    print(line)
    lines_new.append(line)


#!/usr/bin/env bash

# Set walltime limit

#BSUB -W 12:00

#

# Set output file

#BSUB -o 12.107.%I.out

#

# Set error file

#BSUB -eo 12.107.%I.stderr

#

# Specify node group

#BSUB -m "ld-gpu lg-gpu lp-gpu lt-gpu lu-gpu lv-gpu lw-gpu lx-gpu ly-gpu"

#BSUB -q gpuqueue

#

# nodes: number of nodes and GPU request

#BSUB -n 1 -R "rusage[mem=5]"

#BSUB -gpu "num=1:j_exclusive=yes:mode=shared"

#

# job name (default = name of script file)

#BSUB -J "12.107[1-4]"



source ~/.bashrc

module load cuda/10.1

conda activate perses-sims-oct2020



old=50

new=107

old_aa=ASP

new_aa=ALA

resid=39



# Make dirs and copy r-htfs for rest

cd /data/chodera/zhangi/perses_benchmark/neq/12/

cp $old/$old\_apo_0.pickle $new/$new\_apo_0.pickle

cp $old/$old\_apo_1.pickle $new/$new\_apo_1.pickle

cp $old/$old\_complex_0.pickle $new/$new\_complex_0.pickle

cp $old/$old\_complex_1.pickle $new/$new\_complex_1.pickle



# Run and analyze rest

cd /home/zhangi/choderalab/perses_benchmark/perses_protein_m

In [76]:
# Make dir and save new bash file
os.system(f"mkdir {os.path.join(out_dir, str(new))}")

with open(os.path.join(out_dir, str(new), "run_rest2_all.sh"), "w") as f:
     f.writelines(lines_new)
